# Point Cloud Visualization with Axis-Aligned Bounding Box

This notebook demonstrates how to load and visualize a 3D point cloud from a PLY file, along with its axis-aligned bounding box.

## Features:
- Load point cloud from PLY file (both ASCII and binary formats)
- Display point cloud with height-based coloring if no colors are present
- Show axis-aligned bounding box
- Interactive 3D visualization with Open3D
- Point cloud statistics and analysis
- Coordinate frame for reference


In [1]:
# Import required libraries
import open3d as o3d
import numpy as np
import os
import matplotlib.pyplot as plt
from pathlib import Path

# Set up paths
notebook_dir = Path.cwd()
point_cloud_path = notebook_dir / "t_pcd.ply"

print(f"Notebook directory: {notebook_dir}")
print(f"Point cloud path: {point_cloud_path}")
print(f"Point cloud exists: {point_cloud_path.exists()}")


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
Notebook directory: /home/mht/PycharmProjects/pusht_real/test/point_clouds
Point cloud path: /home/mht/PycharmProjects/pusht_real/test/point_clouds/t_pcd.ply
Point cloud exists: True


In [2]:
def load_and_analyze_point_cloud(file_path):
    """
    Load a point cloud from PLY file and return analysis information
    """
    print(f"Loading point cloud from: {file_path}")
    
    # Load the point cloud
    pcd = o3d.io.read_point_cloud(str(file_path))
    
    if len(pcd.points) == 0:
        print("Error: Could not load point cloud or point cloud is empty")
        return None, None
    
    print(f"Point cloud loaded successfully!")
    print(f"Number of points: {len(pcd.points)}")
    print(f"Has colors: {pcd.has_colors()}")
    print(f"Has normals: {pcd.has_normals()}")
    
    # Get point cloud bounds
    bbox = pcd.get_axis_aligned_bounding_box()
    print(f"Bounding box min: {bbox.min_bound}")
    print(f"Bounding box max: {bbox.max_bound}")
    print(f"Bounding box size: {bbox.max_bound - bbox.min_bound}")
    
    # Calculate additional statistics
    points = np.asarray(pcd.points)
    center = np.mean(points, axis=0)
    dimensions = points.max(axis=0) - points.min(axis=0)
    
    print(f"Center: [{center[0]:.4f}, {center[1]:.4f}, {center[2]:.4f}]")
    print(f"Dimensions: [{dimensions[0]:.4f}, {dimensions[1]:.4f}, {dimensions[2]:.4f}]")
    
    return pcd, bbox

# Load and analyze the point cloud
if point_cloud_path.exists():
    pcd, bbox = load_and_analyze_point_cloud(point_cloud_path)
else:
    print(f"Point cloud file not found at {point_cloud_path}")
    pcd, bbox = None, None


Loading point cloud from: /home/mht/PycharmProjects/pusht_real/test/point_clouds/t_pcd.ply
Point cloud loaded successfully!
Number of points: 10000
Has colors: False
Has normals: False
Bounding box min: [-100. -175.    0.]
Bounding box max: [100.  25.  40.]
Bounding box size: [200. 200.  40.]
Center: [-0.7901, -44.3822, 19.7372]
Dimensions: [200.0000, 200.0000, 40.0000]


In [3]:
def add_height_coloring(pcd):
    """
    Add height-based coloring to point cloud if it doesn't have colors
    """
    if pcd.has_colors():
        print("Point cloud already has colors, keeping original colors")
        return pcd
    
    points = np.asarray(pcd.points)
    z_coords = points[:, 2]
    z_min, z_max = z_coords.min(), z_coords.max()
    
    # Normalize Z coordinates to 0-1 range
    z_normalized = (z_coords - z_min) / (z_max - z_min) if z_max > z_min else np.zeros_like(z_coords)
    
    # Create colors based on height (blue to red gradient)
    colors = np.zeros((len(points), 3))
    colors[:, 0] = z_normalized  # Red channel
    colors[:, 2] = 1 - z_normalized  # Blue channel
    
    pcd.colors = o3d.utility.Vector3dVector(colors)
    print("Added height-based coloring to point cloud (blue=low, red=high)")
    return pcd

# Add coloring if needed
if pcd is not None:
    pcd = add_height_coloring(pcd)


Added height-based coloring to point cloud (blue=low, red=high)


In [4]:
def create_bbox_visualization(bbox, color=[1, 0, 0]):
    """
    Create a wireframe visualization of the axis-aligned bounding box
    """
    # Create a wireframe box
    bbox_wireframe = o3d.geometry.LineSet.create_from_axis_aligned_bounding_box(bbox)
    bbox_wireframe.colors = o3d.utility.Vector3dVector([color for _ in range(len(bbox_wireframe.lines))])
    return bbox_wireframe

def create_coordinate_frame(size=0.1):
    """
    Create a coordinate frame for reference
    """
    return o3d.geometry.TriangleMesh.create_coordinate_frame(size=size)

# Create visualization objects
if pcd is not None and bbox is not None:
    # Create bounding box wireframe (red color)
    bbox_wireframe = create_bbox_visualization(bbox, color=[1, 0, 0])
    
    # Create coordinate frame
    coordinate_frame = create_coordinate_frame(size=0.1)
    
    print("Created visualization objects:")
    print(f"- Point cloud: {len(pcd.points)} points")
    print(f"- Bounding box wireframe: {len(bbox_wireframe.lines)} lines")
    print(f"- Coordinate frame: size {0.1}")
else:
    print("Cannot create visualization objects - point cloud not loaded")


Created visualization objects:
- Point cloud: 10000 points
- Bounding box wireframe: 12 lines
- Coordinate frame: size 0.1


## 3D Visualization

The following cell will open an interactive 3D visualization window showing:
- **Point cloud** (colored by height if no original colors)
- **Red wireframe bounding box** (axis-aligned)
- **Coordinate frame** (X=red, Y=green, Z=blue)

### Controls:
- **Mouse**: Rotate view
- **Mouse wheel**: Zoom in/out  
- **Right mouse + drag**: Pan
- **Press 'R'**: Reset view
- **Press 'Q' or close window**: Exit


In [5]:
# Interactive 3D visualization
if pcd is not None and bbox is not None:
    print("Opening 3D visualization window...")
    print("Close the window when done viewing.")
    
    # Combine all visualization objects
    geometries = [pcd, bbox_wireframe, coordinate_frame]
    
    # Start the visualization
    o3d.visualization.draw_geometries(
        geometries,
        window_name="Point Cloud with Axis-Aligned Bounding Box",
        width=1200, 
        height=800,
        left=50,
        top=50
    )
    
    print("3D visualization closed.")
else:
    print("Cannot visualize - point cloud not loaded")


Opening 3D visualization window...
Close the window when done viewing.
3D visualization closed.
